In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import hashlib
import json
import random
import re

# Load raw data - cleaning
---



## Winlogbeat

In [ ]:
raw_data_dataframe1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Resource/elastic_winlogbeat_mar27_to_apr18_merge.xlsx', index_col=0,na_values='-', dtype=str)
raw_data_dataframe2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Resource/elastic_winlogbeat_apr18.xlsx', index_col=0,na_values='-', dtype=str) 

df_list = [raw_data_dataframe1, raw_data_dataframe2]
raw_data_dataframe = pd.concat(df_list)

In [ ]:
# Remove duplicated columns with "keyword" in the name
raw_data_dataframe.drop([col for col in raw_data_dataframe.columns if 'keyword' in col],axis=1,inplace=True)
raw_data_dataframe['winlog.process.pid']=raw_data_dataframe['winlog.process.pid'].str.replace('.','')
raw_data_dataframe=raw_data_dataframe.replace('_x000D_', '',regex=True)
raw_data_dataframe.replace(to_replace=[r"\t+", "\.\n+|\. \n+", "\:\n+|\: \n+"], value=["",". ",": "], regex=True, inplace=True) #whitespace removing
raw_data_dataframe.replace(to_replace=[r"\n+"], value=[". "], regex=True, inplace=True) #whitespace removing



raw_data_dataframe

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


,@version,_id,_index,_score,_type,agent.ephemeral_id,agent.id,agent.name,agent.type,agent.version,...,winlog.event_data.PreAuthType,winlog.event_data.RestrictedAdminMode,winlog.event_data.TargetInfo,winlog.event_data.TargetLinkedLogonId,winlog.event_data.TargetLogonGuid,winlog.event_data.TargetLogonId,winlog.event_data.TargetOutboundDomainName,winlog.event_data.TargetOutboundUserName,winlog.event_data.TargetServerName,winlog.event_data.VirtualAccount
@timestamp,,,,,,,,,,,,,,,,,,,,,
"Apr 17, 2022 @ 23:56:30.512",1.0,Mnr5OYABmEazaiu3Wmt0,winlogbeat,NaN,NaN,192b2156-8ab9-4211-955b-0f807e2b1cff,9e448ec3-a416-4bce-bc2a-73bed1c4ec85,winlogbeat-shipper,winlogbeat,8.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 17, 2022 @ 22:56:30.158",1.0,BnrCOYABmEazaiu3Z2iz,winlogbeat,NaN,NaN,192b2156-8ab9-4211-955b-0f807e2b1cff,9e448ec3-a416-4bce-bc2a-73bed1c4ec85,winlogbeat-shipper,winlogbeat,8.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 17, 2022 @ 22:11:59.071",1.0,y3qZOYABmEazaiu3qWUO,winlogbeat,NaN,NaN,192b2156-8ab9-4211-955b-0f807e2b1cff,9e448ec3-a416-4bce-bc2a-73bed1c4ec85,winlogbeat-shipper,winlogbeat,8.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 17, 2022 @ 22:02:01.990",1.0,Q3qQOYABmEazaiu3iWXR,winlogbeat,NaN,NaN,192b2156-8ab9-4211-955b-0f807e2b1cff,9e448ec3-a416-4bce-bc2a-73bed1c4ec85,winlogbeat-shipper,winlogbeat,8.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 17, 2022 @ 22:00:00.849",1.0,E3qOOYABmEazaiu3r2XH,winlogbeat,NaN,NaN,192b2156-8ab9-4211-955b-0f807e2b1cff,9e448ec3-a416-4bce-bc2a-73bed1c4ec85,winlogbeat-shipper,winlogbeat,8.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Apr 18, 2022 @ 12:15:06.542",1,cnqdPIABmEazaiu3lpKc,winlogbeat,NaN,NaN,192b2156-8ab9-4211-955b-0f807e2b1cff,9e448ec3-a416-4bce-bc2a-73bed1c4ec85,winlogbeat-shipper,winlogbeat,8.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 18, 2022 @ 12:14:40.803",1,cXqdPIABmEazaiu3OZL9,winlogbeat,NaN,NaN,192b2156-8ab9-4211-955b-0f807e2b1cff,9e448ec3-a416-4bce-bc2a-73bed1c4ec85,winlogbeat-shipper,winlogbeat,8.1.1,...,NaN,NaN,NaN,NaN,NaN,33742894,NaN,NaN,NaN,NaN
"Apr 18, 2022 @ 12:14:40.802",1,b3qdPIABmEazaiu3OZL9,winlogbeat,NaN,NaN,192b2156-8ab9-4211-955b-0f807e2b1cff,9e448ec3-a416-4bce-bc2a-73bed1c4ec85,winlogbeat-shipper,winlogbeat,8.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data cleaning

In [ ]:
raw_data_dataframe.dropna(how='all', axis='columns', inplace=True)
raw_data_dataframe.drop(columns=['@version', '_index',	'agent.ephemeral_id',	'agent.id',	'agent.name',	'agent.type',	'agent.version',	'ecs.version', 'winlog.api', 'winlog.version'], inplace=True)
raw_data_dataframe.T.drop_duplicates().T

,_id,event.action,event.code,event.created,event.kind,event.original,event.outcome,event.provider,host.architecture,host.hostname,...,winlog.event_data.ElevatedToken,winlog.event_data.ImpersonationLevel,winlog.event_data.LogonGuid,winlog.event_data.NumberOfGroupPolicyObjects,winlog.event_data.PreAuthType,winlog.event_data.TargetInfo,winlog.event_data.TargetLinkedLogonId,winlog.event_data.TargetLogonGuid,winlog.event_data.TargetLogonId,winlog.event_data.VirtualAccount
@timestamp,,,,,,,,,,,,,,,,,,,,,
"Apr 17, 2022 @ 23:56:30.512",Mnr5OYABmEazaiu3Wmt0,Directory Service Access,4662.0,"Apr 17, 2022 @ 23:56:32.079",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,regina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 17, 2022 @ 22:56:30.158",BnrCOYABmEazaiu3Z2iz,Directory Service Access,4662.0,"Apr 17, 2022 @ 22:56:31.177",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,regina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 17, 2022 @ 22:11:59.071",y3qZOYABmEazaiu3qWUO,Kerberos Service Ticket Operations,4770.0,"Apr 17, 2022 @ 22:12:01.067",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,regina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 17, 2022 @ 22:02:01.990",Q3qQOYABmEazaiu3iWXR,NaN,7040.0,"Apr 17, 2022 @ 22:02:03.275",event,<Event xmlns='http://schemas.microsoft.com/win...,NaN,Service Control Manager,x86_64,regina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 17, 2022 @ 22:00:00.849",E3qOOYABmEazaiu3r2XH,Security Group Management,4799.0,"Apr 17, 2022 @ 22:00:01.925",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,regina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Apr 18, 2022 @ 12:15:06.542",cnqdPIABmEazaiu3lpKc,NaN,7036,"Apr 18, 2022 @ 12:15:07.577",event,<Event xmlns='http://schemas.microsoft.com/win...,NaN,Service Control Manager,x86_64,regina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Apr 18, 2022 @ 12:14:40.803",cXqdPIABmEazaiu3OZL9,Logoff,4634,"Apr 18, 2022 @ 12:14:42.477",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,regina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33742894,NaN
"Apr 18, 2022 @ 12:14:40.802",b3qdPIABmEazaiu3OZL9,Special Logon,4672,"Apr 18, 2022 @ 12:14:42.477",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,regina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
raw_data_dataframe.drop_duplicates('message', inplace=True)
raw_data_dataframe.reset_index(drop=True, inplace=True)

In [ ]:
raw_data_dataframe['@timestamp']=raw_data_dataframe['event.original'].str.extract("<TimeCreated SystemTime='(.*)'\/><Event")

In [ ]:
raw_data_dataframe['@timestamp']

0       2022-04-17T23:56:30.512220700Z
1       2022-04-17T22:11:59.071084400Z
2       2022-04-17T22:02:01.990230100Z
3       2022-04-17T22:00:00.849971500Z
4       2022-04-17T22:00:00.848104700Z
                     ...              
9095    2022-04-18T12:15:06.734886600Z
9096    2022-04-18T12:14:40.803929400Z
9097    2022-04-18T12:14:40.802387000Z
9098    2022-04-18T12:14:40.802540500Z
9099    2022-04-18T12:13:40.798951600Z
Name: @timestamp, Length: 9100, dtype: object

## Fortigate

In [ ]:
fortigate_dataframe = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Resource/fortigate_firewall_min.csv', sep=',', dtype=str, na_values='')
# Remove duplicated columns with "keyword" in the name
fortigate_dataframe.dropna(how='all', axis='columns', inplace=True)

fortigate_dataframe

,Event Time,Receipt Time,Device,Logger,Version,deviceVendor,deviceProduct,deviceVersion,deviceEventClassId,name,...,ad.FTNTFGTdstdevtype,ad.FTNTFGTdstmac,ad.FTNTFGTmasterdstmac,ad.FTNTFGTdstosname,destinationTranslatedAddress,destinationTranslatedZoneURI,destinationTranslatedPort,ad.FTNTFGTdstserver,ad.FTNTFGTdstosversion,ad.FTNTFGTdstdevcategory
0,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17379,2022/04/21 17:09:59 CEST,2022/04/21 17:10:05 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward close,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17380,2022/04/21 17:09:59 CEST,2022/04/21 17:10:05 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward close,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17381,2022/04/21 17:09:59 CEST,2022/04/21 17:10:05 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward close,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17382,2022/04/21 17:09:59 CEST,2022/04/21 17:10:05 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward close,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
fortigate_dataframe.drop_duplicates(['sourceHostName','destinationAddress','name','deviceEventClassId', 'ad.FTNTFGTduration'], inplace=True)

In [ ]:
fortigate_dataframe

,Event Time,Receipt Time,Device,Logger,Version,deviceVendor,deviceProduct,deviceVersion,deviceEventClassId,name,...,ad.FTNTFGTdstdevtype,ad.FTNTFGTdstmac,ad.FTNTFGTmasterdstmac,ad.FTNTFGTdstosname,destinationTranslatedAddress,destinationTranslatedZoneURI,destinationTranslatedPort,ad.FTNTFGTdstserver,ad.FTNTFGTdstosversion,ad.FTNTFGTdstdevcategory
0,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022/04/21 17:00:07 CEST,2022/04/21 17:00:10 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward accept,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17377,2022/04/21 17:09:59 CEST,2022/04/21 17:10:05 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward close,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17378,2022/04/21 17:09:59 CEST,2022/04/21 17:10:05 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward close,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17379,2022/04/21 17:09:59 CEST,2022/04/21 17:10:05 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward close,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17380,2022/04/21 17:09:59 CEST,2022/04/21 17:10:05 CEST,vm-rs-01.jcekb.cz,Local,0,Fortinet,Fortigate,v6.0.9,00013,traffic:forward close,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Dictionary of metakey and associated question

---



In [ ]:
metakey_questions = {
    "app.category" : ["What is the category of application?"],
    #"bytes.in" : ["What is IN value?"],#["How many bytes are coming in?"],
    #"bytes.out" : ["What is OUT value?"],#["How many bytes are coming out?"],
    "country.dst" : ["What is the destination country?"],
    "country.src" : ["What is the source country?"],
    "dns.class" : ["What is the class of DNS record?"],
    "dns.origin.ip" : ["Which IP address is origin of the DNS event?"],
    "dns.server" : ["What is IP address of DNS server (DnsServerList)?"],
    "dns.suffix" : ["What is domain suffix?"],
    "dns.type" : ["What is the type of DNS record"],
    "error.code" : ["What is the error code?"],
    "event.action" : ["What is the device's event action?"],
    "event.category": ["What is the event category?"],
    "event.channel" : ["Which event channel is the source?"],
    "event.commandline" : ["Which command was used?"],
    "event.id" : ["What is the event id (EventID or logid)?"],
    "event.operation" : ['Which operation or task was provided?'],
    "event.provider" : ['Which process is a provider of event information?'],
    #"file.path" : ["What is related file path?"],
    "host.domain" : ["What is the host's domain name?"],
    "host.external.id" : ["What is the devices's external id (ExternalId)?"],
    #"host.hostname" : ["What is the name of the host origin?"],
    "host.hostname" : ["What is the name of the source host?"],
    #"host.remote" : ["What is the name of related remote host?"],
    "int.dst.role" : ["What is the role of destination interface?"],
    "int.inbound" : ["What si the name of the inbound interface?"],
    "int.outbound" : ["What is the name of the outbound interface?"],
    "int.src.role" : ["What is the role of source interface?"],
    "ip.dst" : ["What is the destination IP address?"],
    "ip.src" : ["What is the source IP address?"],
    "ip.translated" : ["What is the translated source IP address?"],
    "loglevel" : ["What is the level of log?"],
    "logon.type" : ["Which logon type (LogonType) was used?"],
    "mac.src" : ["What is the source MAC address?"],
    "message" : ["What is the message of the event?"],
    "OS.machine" : ["What is the OS name of the source device?"],
    "policy.id" : ["What is the id of policy (policyid)?"],
    "policy.type" : ["What is the type of policy (policytype)?"],
    "port.dst" : ["What is the destination port (dpt)?"],
    "port.src" : ["What is the source port (spt)?"],
    "port.translated" : ["What is the translated source port?"],
    "process.name" : ["Which process is involved?"],
    "process.pid" : ["What is PID of responsible process?"],
    "protocol" : ["Which application protocol is used?"],
    "timestamp" : ["When did the event happen?"],
    "user.origin.id" : ["What is the identification number of the origin user?"],
    "user.origin.name" : ["Which user is the origin of the event?"],
    "user.target.id" : ["What is the identification number of the target user?"],
    "user.target.name" : ["Which user is the target of the event?"]
}

for key in metakey_questions:
    metakey_questions[key].append(hashlib.md5(metakey_questions[key][0].encode()).hexdigest())

In [ ]:
#winlogbeat
mapped_metakey = {
    "dns.origin.ip" : ["winlog.event_data.Ipaddress"],
    "dns.server" : ["winlog.event_data.DnsServerList"],
    "dns.suffix" : ["winlog.event_data.AdapterSuffixName"],
    "event.channel" : ["winlog.channel"],
    "event.commandline" : ["winlog.event_data.AdapterName"],
    "event.id" : ["event.code"],
    "event.operation" : ["winlog.task", "winlog.event_data.Operation"], #"winlog.event_data.AuthenticationPackageName"
    "event.provider" : ["winlog.provider_name"],
    "host.domain" : ["winlog.event_data.SubjectDomainName"],
    "host.hostname" : ["host.name"],
    "loglevel" : ["log.level"],
    #"logon.type" : ["winlog.event_data.LogonProcessName", "winlog.event_data.LogonType"],
    #"message" : ["message"],
    "process.name" : ["winlog.event_data.ProcessName", "winlog.event_data.CallerProcessName", "winlog.event_data.DisplayName", "winlog.event_data.LogonProcessName"],
    "process.pid" : ["winlog.process.pid"], #"winlog.event_data.ProcessId", "winlog.event_data.CallerProcessId"
    "timestamp" : ["@timestamp"],
    "user.origin.id" : ["winlog.event_data.SubjectUserSid"],
    "user.origin.name" : ["winlog.event_data.SubjectUserName"], #"winlog.user.domain", "winlog.user.name"
    "user.target.id" : ["winlog.event_data.TargetUserSid"],
    "user.target.name" : ["winlog.event_data.TargetUserName"]
    #"error.code" : ["winlog.event_data.ErrorCode"]
    #"file.path" : [],
    #"host.remote" : [],
    #"ip.src" : [],
    #"OS.machine" : ["host.os.name"],
}

In [ ]:
#fortigate
mapped_metakey = {
    "app.category" : ["ad.FTNTFGTappcat"],
    #"bytes.in" : ["bytesIn"],
    #"bytes.out" : ["bytesOut"],
    "country.dst" : ["ad.FTNTFGTdstcountry"],
    "country.src" : ["ad.FTNTFGTsrccountry"],
    "dns.class" : ["ad.FTNTFGTqclass"],
    "dns.type" : ["ad.FTNTFGTqtype"],
    "error.code" : ["ad.FTNTFGTerror"],
    "event.action" : ["deviceAction"],
    "event.category" : ["deviceEventCategory"],
    "event.id" : ["ad.FTNTFGTlogid"],
    "host.external.id" : ["deviceExternalId"],
    "host.hostname" : ["sourceHostName"],
    "int.dst.role" : ["ad.FTNTFGTdstintfrole"],
    "int.inbound" : ["deviceInboundInterface"],
    "int.outbound" : ["deviceOutboundInterface"],
    "int.src.role" : ["ad.FTNTFGTsrcintfrole"],
    "ip.dst" : ["destinationAddress"],
    "ip.src" : ["sourceAddress"],
    "ip.translated" : ["sourceTranslatedAddress"],
    "loglevel" : ["ad.FTNTFGTlevel"],
    "mac.src" : ["ad.FTNTFGTsrcmac"],
    "message" : ["message"],
    "OS.machine" : ["ad.FTNTFGTosname"],
    "policy.id" : ["ad.FTNTFGTpolicyid"],
    "policy.type" : ["ad.FTNTFGTpolicytype"],
    "port.dst" : ["destinationPort"],
    "port.src" : ["sourcePort"],
    "port.translated" : ["sourceTranslatedPort"],
    "protocol" : ["applicationProtocol"],
    "timestamp" : ["deviceReceiptTime"]
}



# Final structure

---



## Winlogbeat

In [ ]:
#dataframe_grouped = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Resource/dataframe_winlogbean_Mar27_to_Apr18.xlsx', index_col=0, na_values='NaN', dtype={'winlog.event_data.CallerProcessId':str, 'winlog.process.pid':str})  

#dataframe_grouped = raw_data_dataframe.groupby('event.code')
dataframe_grouped = raw_data_dataframe


In [ ]:
#dataframe_grouped['@timestamp'] = dataframe_grouped['@timestamp'].replace({' @':''}, regex=True)
#dataframe_grouped['@timestamp'] = pd.to_datetime(dataframe_grouped['@timestamp'])
#dataframe_grouped['@timestamp'] = dataframe_grouped['@timestamp'].dt.strftime("%Y-%m-%dT%H:%M:%S.%f").str[:-3]
gr = dataframe_grouped.groupby('event.code')

In [ ]:
dataframe_grouped

,@timestamp,_id,event.action,event.code,event.created,event.kind,event.original,event.outcome,event.provider,host.architecture,...,winlog.event_data.ImpersonationLevel,winlog.event_data.LogonGuid,winlog.event_data.NumberOfGroupPolicyObjects,winlog.event_data.PreAuthType,winlog.event_data.TargetInfo,winlog.event_data.TargetLinkedLogonId,winlog.event_data.TargetLogonGuid,winlog.event_data.TargetLogonId,winlog.event_data.TargetServerName,winlog.event_data.VirtualAccount
0,2022-04-17T23:56:30.512,Mnr5OYABmEazaiu3Wmt0,Directory Service Access,4662.0,"Apr 17, 2022 @ 23:56:32.079",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-04-17T22:11:59.071,y3qZOYABmEazaiu3qWUO,Kerberos Service Ticket Operations,4770.0,"Apr 17, 2022 @ 22:12:01.067",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-04-17T22:02:01.990,Q3qQOYABmEazaiu3iWXR,NaN,7040.0,"Apr 17, 2022 @ 22:02:03.275",event,<Event xmlns='http://schemas.microsoft.com/win...,NaN,Service Control Manager,x86_64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-04-17T22:00:00.849,E3qOOYABmEazaiu3r2XH,Security Group Management,4799.0,"Apr 17, 2022 @ 22:00:01.925",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-04-17T22:00:00.848,EnqOOYABmEazaiu3r2XH,Security Group Management,4799.0,"Apr 17, 2022 @ 22:00:01.925",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9095,2022-04-18T12:15:06.734,c3qdPIABmEazaiu3mpKM,Special Logon,4672,"Apr 18, 2022 @ 12:15:08.643",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9096,2022-04-18T12:14:40.803,cXqdPIABmEazaiu3OZL9,Logoff,4634,"Apr 18, 2022 @ 12:14:42.477",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33742894,NaN,NaN
9097,2022-04-18T12:14:40.802,b3qdPIABmEazaiu3OZL9,Special Logon,4672,"Apr 18, 2022 @ 12:14:42.477",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9098,2022-04-18T12:14:40.802,cHqdPIABmEazaiu3OZL9,Logon,4624,"Apr 18, 2022 @ 12:14:42.477",event,<Event xmlns='http://schemas.microsoft.com/win...,success,Microsoft-Windows-Security-Auditing,x86_64,...,%%1833,{50B1AC1F-B643-098B-2014-8CE3AB3D5B69},NaN,NaN,NaN,0,NaN,33742894,NaN,%%1843


### Final dataset structure - winlog


In [ ]:
output = []


for x in gr.groups:
  group_x = gr.get_group(x)
  group_x.dropna(how='all', axis='columns', inplace=True)
  #print(x, group_x.columns)

  for map_key in mapped_metakey.keys():
    for map_value in mapped_metakey[map_key]:
      if map_value in group_x.columns:
        for index, log in group_x.iterrows():
          if (str(log[map_value]).lower() != 'nan'):
            answer = str(log[map_value])
            ans_start_log = str(log['event.original']).lower().replace(' \n','').replace('\n','').replace('\n ','')
            ans_start_ex = answer.lower().replace(' \n','').replace('\n','').replace('\n ','')

            if map_key == 'message':
              answer_start = ans_start_log.find(ans_start_ex[:20])
            else:
              answer_start = ans_start_log.find(ans_start_ex)
            if answer_start == -1:
              continue

            result = dict()
            result.update({
              'log_id' : hashlib.md5(log['_id'].encode()).hexdigest(),
              'raw_data' : str(log['event.original']),
              'question' : metakey_questions[map_key][0],
              'question_id' : metakey_questions[map_key][1],
              'answer': answer,
              'answer_start' : answer_start

            })
            output.append(result)

print(output[:4])    
random.shuffle(output)


train = output[:int(len(output)*0.6)]
valid = output[int(len(output)*0.6):int(len(output)*0.8)]
test = output[int(len(output)*0.8):]


print("Dataset statistics")
print("Dataset:", len(output))
print("Train Set:", len(train))
print("Validation Set: ", len(valid))
print('Test Set: ', len(test))

dataset={
    'train' : train,
    'valid' : valid,
}



with open('drive/MyDrive/Colab Notebooks/Resource/dataset_winlog_train.json', 'w') as outfile:
    outfile.write(json.dumps(train[0:30000]))
    #outfile.write(json.dumps(train))

with open('drive/MyDrive/Colab Notebooks/Resource/dataset_winlog_valid.json', 'w') as outfile:
    outfile.write(json.dumps(valid[0:10000]))
    #outfile.write(json.dumps(valid))

with open('drive/MyDrive/Colab Notebooks/Resource/dataset_winlog_test.json', 'w') as outfile:
    outfile.write(json.dumps(valid[0:10000]))
    #outfile.write(json.dumps(test))

with open('drive/MyDrive/Colab Notebooks/Resource/dataset_winlog.json', 'w') as outfile:
    outfile.write(json.dumps(dataset))

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


[{'log_id': '348f614a21933ee6d71662e4dd11a41f', 'raw_data': "<Event xmlns='http://schemas.microsoft.com/win/2004/08/events/event'><System><Provider Name='edgeupdate'/><EventID Qualifiers='0'>0</EventID><Version>0</Version><Level>4</Level><Task>0</Task><Opcode>0</Opcode><Keywords>0x80000000000000</Keywords><TimeCreated SystemTime='2022-04-16T20:48:15.5358441Z'/><EventRecordID>2741</EventRecordID><Correlation/><Execution ProcessID='0' ThreadID='0'/><Channel>Application</Channel><Computer>radka.vmware.fekt.cz</Computer><Security/></System><EventData><Data>Service stopped</Data></EventData><RenderingInfo Culture='en-US'><Message>Service stopped.</Message><Level>Information</Level><Task></Task><Opcode>Info</Opcode><Channel></Channel><Provider></Provider><Keywords><Keyword>Classic</Keyword></Keywords></RenderingInfo></Event>", 'question': 'Which event channel is the source?', 'question_id': '38ad395fbea1fea22bb49476b71904d6', 'answer': 'Application', 'answer_start': 399}, {'log_id': '348f614

## Fortigate

In [ ]:
fortigate_dataframe_grouped = fortigate_dataframe

In [ ]:
gr = fortigate_dataframe_grouped.groupby('ad.FTNTFGTlogid')

### Final dataset structure - fortigate


In [ ]:
output = []


for x in gr.groups:
  group_x = gr.get_group(x)
  group_x.dropna(how='all', axis='columns', inplace=True)
  #print(x, group_x.columns)

  for map_key in mapped_metakey.keys():
    for map_value in mapped_metakey[map_key]:
      if map_value in group_x.columns:
        for index, log in group_x.iterrows():
          if (str(log[map_value]).lower() != 'nan'):
            answer = str(log[map_value])
            ans_start_log = str(log['rawEvent']).lower().replace(' \n','').replace('\n','').replace('\n ','')
            ans_start_ex = answer.lower().replace(' \n','').replace('\n','').replace('\n ','')

            answer_start = ans_start_log.find(ans_start_ex)
            if answer_start == -1:
              continue

            result = dict()
            result.update({
              'log_id' : hashlib.md5(log['rawEvent'].encode()).hexdigest(),
              'raw_data' : str(log['rawEvent']),
              'question' : metakey_questions[map_key][0],
              'question_id' : metakey_questions[map_key][1],
              'answer': answer,
              'answer_start' : answer_start

            })
            output.append(result)

print(output[:4])    
random.shuffle(output)


train = output[:int(len(output)*0.6)]
valid = output[int(len(output)*0.6):int(len(output)*0.8)]
test = output[int(len(output)*0.8):]


print("Dataset statistics")
print("Dataset:", len(output))
print("Train Set:", len(train))
print("Validation Set: ", len(valid))
print('Test Set: ', len(test))

dataset={
    'train' : train,
    'valid' : valid,
}



with open('drive/MyDrive/Colab Notebooks/Resource/dataset_fortigate_train.json', 'w') as outfile:
    outfile.write(json.dumps(train[0:30000]))
    #outfile.write(json.dumps(train))

with open('drive/MyDrive/Colab Notebooks/Resource/dataset_fortigate_valid.json', 'w') as outfile:
    outfile.write(json.dumps(valid[0:10000]))
    #outfile.write(json.dumps(valid))

with open('drive/MyDrive/Colab Notebooks/Resource/dataset_fortigate_test.json', 'w') as outfile:
    outfile.write(json.dumps(valid[0:10000]))
    #outfile.write(json.dumps(test))

with open('drive/MyDrive/Colab Notebooks/Resource/dataset_fortigate.json', 'w') as outfile:
    outfile.write(json.dumps(dataset))

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


[{'log_id': '85c1b3ec076cacefb9f45c155ff8b38b', 'raw_data': '<117>Apr 21 17:02:59 172.25.164.254 CEF:0|Fortinet|Fortigate|v6.0.9|00013|traffic:forward accept|3|deviceExternalId=FG5H0E5819908089 FTNTFGTlogid=0000000013 cat=traffic:forward FTNTFGTsubtype=forward FTNTFGTlevel=notice FTNTFGTvd=vlab FTNTFGTeventtime=1650553379 src=172.30.104.1 shost=WIN-7OCV2UEAUKA spt=55095 deviceInboundInterface=vmservers FTNTFGTsrcintfrole=lan dst=192.33.14.30 dpt=53 deviceOutboundInterface=root-vlab1 FTNTFGTdstintfrole=undefined FTNTFGTpoluuid=d0d4bdd2-4d95-51ea-f827-483b4dae641a externalId=3341608935 proto=17 act=accept FTNTFGTpolicyid=7 FTNTFGTpolicytype=policy app=DNS FTNTFGTdstcountry=United States FTNTFGTsrccountry=Reserved FTNTFGTtrandisp=snat sourceTranslatedAddress=84.19.71.108 sourceTranslatedPort=55095 FTNTFGTappid=16195 FTNTFGTapp=DNS FTNTFGTappcat=Network.Service FTNTFGTapprisk=elevated FTNTFGTapplist=g-default FTNTFGTduration=180 out=79 in=696 FTNTFGTsentpkt=1 FTNTFGTrcvdpkt=1 FTNTFGTdevtyp

# *** Semestralna praca ***

---
## Apache logs transformed from csv to json
---

In [ ]:
import hashlib
import json  
import csv

path = "drive/MyDrive/Colab Notebooks/Resource/apache.csv"


output = []
with open(path) as csv_file:
  csv_file.readline()
  for log in csv.reader(csv_file):
    #print(log)
    parsed_data = {
        'timestamp' : log[0],
        '_index' : log[2],
        '_score' : log[3],
        '_type' : log[4],
        'ip.src' : log[5],
        'loglevel' : log[7],
        'message' : log[6]
        }
    output.append({
        'log_id' : hashlib.md5(log[1].encode()).hexdigest(),
        'raw_data' : log[8],
        'parsed_data' : parsed_data
    })

print(len(output))

with open('drive/MyDrive/Colab Notebooks/Resource/ApacheJsonFile.json', 'w') as outfile:
    outfile.write(json.dumps(output))

31


---
## Linux logs transformed from csv to json
---

In [ ]:
import hashlib
import json  
import csv

path = "drive/MyDrive/Colab Notebooks/Resource/linux.csv"


output = []
with open(path) as csv_file:
  csv_file.readline()
  for log in csv.reader(csv_file):
    #print(log)
    parsed_data = {
        'timestamp' : log[15],
        '_index' : log[2],
        '_score' : log[3],
        '_type' : log[4],
        'event.kind' : log[5],
        'host.hostname' : log[6],
        'message' : log[7],
        "process.name" : log[8],
        "process.pid" : log[9],
        "related.hosts" : log[10],
        "related.user" : log[11],
        "user.effective.name" : log[12],
        "user.id" : log[13],
        "user.name" : log[14]
        }
    output.append({
        'log_id' : hashlib.md5(log[1].encode()).hexdigest(),
        'raw_data' : log[16],
        'parsed_data' : parsed_data
      
    })

print(len(output))


with open('drive/MyDrive/Colab Notebooks/Resource/LinuxJsonFile.json', 'w') as outfile:
    outfile.write(json.dumps(output))

17


---
## Json for training a model - Apache
---

In [ ]:
import hashlib
import json  
import csv
import random

apache_questions = ['When did the event happen?',
                    'Which machine is the source of the log event?',
                    'What is the source IP address?',
                    'What is the log level?',
                    'What happened?'
]

output = []


with open('drive/MyDrive/Colab Notebooks/Resource/ApacheJsonFile.json', 'r') as file:
  apache_data = json.load(file)
  for a in apache_data:
    for q in range(len(apache_questions)):
      result = dict()
      result.update({
        'log_id' : a['log_id'],
        'raw_data' : a['raw_data'],
        'question' : apache_questions[q],
        'question_id' : hashlib.md5(apache_questions[q].encode()).hexdigest()
      })
      if q == 0:
        result.update({
          'answer': a['parsed_data']['timestamp'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['timestamp'])#+1
        })
      elif q == 1:
        ans = (a['parsed_data']['_index'])[:6]
        result.update({
          'answer': ans,
          'answer_start' : (a['raw_data']).find(ans)#+1
        })
      elif q == 2:
        result.update({
          'answer': a['parsed_data']['ip.src'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['ip.src'])#+1
        })
      elif q == 3:
        result.update({
          'answer': a['parsed_data']['loglevel'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['loglevel'])#+1
        })
      elif q == 4:
        result.update({
          'answer': a['parsed_data']['message'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['message'])#+1
        })
      
      output.append(result)

print(output[:4])

random.shuffle(output)


with open('drive/MyDrive/Colab Notebooks/Resource/TrainingJsonFile_apache.json', 'w') as outfile:
    outfile.write(json.dumps(output))

[{'log_id': '5aedf0a485895504d9dcbfa28bd98a19', 'raw_data': '[Thu Jun 09 07:11:21 2005] [error] [client 204.100.200.22] Directory index forbidden by rule: /var/www/html/', 'question': 'When did the event happen?', 'question_id': 'd9694423546492bc6ab2803e50d2b8b0', 'answer': 'Thu Jun 09 07:11:21 2005', 'answer_start': 1}, {'log_id': '5aedf0a485895504d9dcbfa28bd98a19', 'raw_data': '[Thu Jun 09 07:11:21 2005] [error] [client 204.100.200.22] Directory index forbidden by rule: /var/www/html/', 'question': 'Which machine is the source of the log event?', 'question_id': 'd277c9c87723d66677d93a8fe1428b95', 'answer': 'apache', 'answer_start': -1}, {'log_id': '5aedf0a485895504d9dcbfa28bd98a19', 'raw_data': '[Thu Jun 09 07:11:21 2005] [error] [client 204.100.200.22] Directory index forbidden by rule: /var/www/html/', 'question': 'What is the source IP address?', 'question_id': '75c2dc5ad90e4b934aedb01a3453b0e8', 'answer': '204.100.200.22', 'answer_start': 43}, {'log_id': '5aedf0a485895504d9dcbfa2

---
##Json for training a model - Linux
---

In [ ]:
import hashlib
import json  
import csv

linux_questions = ['When did the event happen?',
                   'Which machine is the source of the log event?',
                   'What is the name of the host?',
                   'What happened?',
                   'Which process is responsible?',
                   'What is PID of the responsible process?',
                   'What is the name of related host?',
                   #'Is there any related user with the issue?',
                   'What is the name of related user?',
                   'What is the user id?'
]

output = []


with open('drive/MyDrive/Colab Notebooks/Resource/LinuxJsonFile.json', 'r') as file:
  linux_data = json.load(file)
  for a in linux_data:
    for q in range(len(linux_questions)):
      result = dict()
      result.update({
        'log_id' : a['log_id'],
        'raw_data' : a['raw_data'],
        'question' : linux_questions[q],
        'question_id' : hashlib.md5(linux_questions[q].encode()).hexdigest()
      })
      if q == 0:
        result.update({
          'answer': a['parsed_data']['timestamp'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['timestamp'])#+1
        })
      elif q == 1:
        ans = (a['parsed_data']['_index'])[:5]
        result.update({
          'answer': ans,
          'answer_start' : (a['raw_data']).find(ans)#+1
        })
      elif q == 2:
        result.update({
          'answer': a['parsed_data']['host.hostname'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['host.hostname'])#+1
        })
      elif q == 3:
        result.update({
          'answer': a['parsed_data']['message'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['message'])#+1
        })
      elif q == 4:
        result.update({
          'answer': a['parsed_data']['process.name'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['process.name'])#+1
        })
      elif q == 5:
        result.update({
          'answer': a['parsed_data']['process.pid'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['process.pid'])#+1
        })
      elif q == 6:
        result.update({
          'answer': a['parsed_data']['related.hosts'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['related.hosts'])#+1
        })
      elif q == 7:
        #ans = ''
        #if a['parsed_data']['related.user'] != '-':
        #  ans = 'Yes'
        #else:
        #  ans = 'No'
        result.update({
          'answer': a['parsed_data']['related.user'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['related.user'])#+1
        })
      elif q == 8:
        #ans = ''
        #if a['parsed_data']['related.user'] != '-':
        #  ans = 'Yes'
        #else:
        #  ans = 'No'
        result.update({
          'answer': a['parsed_data']['user.id'],
          'answer_start' : (a['raw_data']).find(a['parsed_data']['user.id'])#+1
        })
      output.append(result)

print(output[:4])

random.shuffle(output)

with open('drive/MyDrive/Colab Notebooks/Resource/TrainingJsonFile_linux.json', 'w') as outfile:
    outfile.write(json.dumps(output))

[{'log_id': 'f40b1beb7217df0588e1be36939e4915', 'raw_data': 'Dec 31 04:07:33 combo su(pam_unix)[2919]: session opened for user news by (uid=0)', 'question': 'When did the event happen?', 'question_id': 'd9694423546492bc6ab2803e50d2b8b0', 'answer': 'Dec 31 04:07:33', 'answer_start': 0}, {'log_id': 'f40b1beb7217df0588e1be36939e4915', 'raw_data': 'Dec 31 04:07:33 combo su(pam_unix)[2919]: session opened for user news by (uid=0)', 'question': 'Which machine is the source of the log event?', 'question_id': 'd277c9c87723d66677d93a8fe1428b95', 'answer': 'linux', 'answer_start': -1}, {'log_id': 'f40b1beb7217df0588e1be36939e4915', 'raw_data': 'Dec 31 04:07:33 combo su(pam_unix)[2919]: session opened for user news by (uid=0)', 'question': 'What is the name of the host?', 'question_id': 'ed1e3fa9b1e209e09b13ec0c507855d9', 'answer': 'combo', 'answer_start': 16}, {'log_id': 'f40b1beb7217df0588e1be36939e4915', 'raw_data': 'Dec 31 04:07:33 combo su(pam_unix)[2919]: session opened for user news by (ui

---
## Final merge of training dataset - split to training, testing, validation set
---

In [ ]:
import json
import random

with open('drive/MyDrive/Colab Notebooks/Resource/TrainingJsonFile_apache.json', 'r') as file:
  apache_data = json.load(file)

with open('drive/MyDrive/Colab Notebooks/Resource/TrainingJsonFile_linux.json', 'r') as file:
  linux_data = json.load(file)

#apache_data += "\n"
data = apache_data + linux_data

print('Print first 6 items as an example')
print(data[:6])
random.shuffle(data)
random.shuffle(data)
random.shuffle(data)
print('\nPrint first 6 items as a shuffled example')
print(data[:6])

print(len(data))
#json_data = {"data" : data}
#print(type(json_data))

train = data[:int(len(data)*0.6)]
valid = data[int(len(data)*0.6):int(len(data)*0.8)]
test = data[int(len(data)*0.8):]

print('\nSplit dataset:')
print('Train: ', len(train))
print('Valid: ', len(valid))
print('Test: ', len(test))

with open('drive/MyDrive/Colab Notebooks/Resource/TrainingJsonFile_final.json', 'w') as outfile:
    outfile.write(json.dumps(train))
    #outfile.write(json.dumps(json_data))

with open('drive/MyDrive/Colab Notebooks/Resource/ValidationJsonFile.json', 'w') as outfile:
    outfile.write(json.dumps(valid))

with open('drive/MyDrive/Colab Notebooks/Resource/TestingJsonFile.json', 'w') as outfile:
    outfile.write(json.dumps(test))


dataset = {'train' : train,
           'valid' : valid,
           'test' : test
           }

with open('drive/MyDrive/Colab Notebooks/Resource/dataset.json', 'w') as outfile:
    outfile.write(json.dumps(dataset))

Print first 6 items as an example
[{'log_id': '282b0b8385e95a7abfd8c945460e5674', 'raw_data': '[Sat Jun 11 23:39:35 2005] [error] [client 61.10.7.36] Directory index forbidden by rule: /var/www/html/', 'question': 'What is the source IP address?', 'question_id': '75c2dc5ad90e4b934aedb01a3453b0e8', 'answer': '61.10.7.36', 'answer_start': 43}, {'log_id': '290d6a8697ba9a94fd17372a062c1cbd', 'raw_data': '[Tue Nov 29 06:12:13 2005] [error] [client 210.245.233.251] File does not exist: /var/www/html/log', 'question': 'When did the event happen?', 'question_id': 'd9694423546492bc6ab2803e50d2b8b0', 'answer': 'Tue Nov 29 06:12:13 2005', 'answer_start': 1}, {'log_id': '6bb32e34bb7a311c3ba68fac7fa0ef5f', 'raw_data': '[Sun Jun 12 04:04:29 2005] [notice] LDAP: Built with OpenLDAP LDAP SDK', 'question': 'Which machine is the source of the log event?', 'question_id': 'd277c9c87723d66677d93a8fe1428b95', 'answer': 'apache', 'answer_start': -1}, {'log_id': '39180a32b60ed08ef5f40a3b1476a599', 'raw_data':